<a href="https://colab.research.google.com/github/nedokormysh/lin_models_presentation/blob/model_streamlit/project_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практическая работа

# Задача

Один из способов повысить эффективность взаимодействия банка с клиентами — отправлять предложение о новой услуге не всем клиентам, а только некоторым, которые выбираются по принципу наибольшей склонности к отклику на это предложение.

Задача заключается в том, чтобы предложить алгоритм, который будет выдавать склонность клиента к положительному или отрицательному отклику на предложение банка. Предполагается, что, получив такие оценки для некоторого множества клиентов, банк обратится с предложением только к тем, от кого ожидается положительный отклик.


Для решения этой задачи загрузите файлы из базы в Postgres.
Эта БД хранит информацию о клиентах банка и их персональные данные, такие как пол, количество детей и другие.

Описание таблиц с данными представлено ниже.


**D_work**

Описание статусов относительно работы:
- ID — идентификатор социального статуса клиента относительно работы;
- COMMENT — расшифровка статуса.


**D_pens**

Описание статусов относительно пенсии:
- ID — идентификатор социального статуса;
- COMMENT — расшифровка статуса.


**D_clients**

Описание данных клиентов:
- ID — идентификатор записи;
- AGE	— возраст клиента;
- GENDER — пол клиента (1 — мужчина, 0 — женщина);
- EDUCATION — образование;
- MARITAL_STATUS — семейное положение;
- CHILD_TOTAL	— количество детей клиента;
- DEPENDANTS — количество иждивенцев клиента;
- SOCSTATUS_WORK_FL	— социальный статус клиента относительно работы (1 — работает, 0 — не работает);
- SOCSTATUS_PENS_FL	— социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
- REG_ADDRESS_PROVINCE — область регистрации клиента;
- FACT_ADDRESS_PROVINCE — область фактического пребывания клиента;
- POSTAL_ADDRESS_PROVINCE — почтовый адрес области;
- FL_PRESENCE_FL — наличие в собственности квартиры (1 — есть, 0 — нет);
- OWN_AUTO — количество автомобилей в собственности.


**D_agreement**

Таблица с зафиксированными откликами клиентов на предложения банка:
- AGREEMENT_RK — уникальный идентификатор объекта в выборке;
- ID_CLIENT — идентификатор клиента;
- TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было).
    
    
**D_job**

Описание информации о работе клиентов:
- GEN_INDUSTRY — отрасль работы клиента;
- GEN_TITLE — должность;
- JOB_DIR — направление деятельности внутри компании;
- WORK_TIME — время работы на текущем месте (в месяцах);
- ID_CLIENT — идентификатор клиента.


**D_salary**

Описание информации о заработной плате клиентов:
- ID_CLIENT — идентификатор клиента;
- FAMILY_INCOME — семейный доход (несколько категорий);
- PERSONAL_INCOME — личный доход клиента (в рублях).


**D_last_credit**

Информация о последнем займе клиента:
- ID_CLIENT — идентификатор клиента;
- CREDIT — сумма последнего кредита клиента (в рублях);
- TERM — срок кредита;
- FST_PAYMENT — первоначальный взнос (в рублях).


**D_loan**

Информация о кредитной истории клиента:
- ID_CLIENT — идентификатор клиента;
- ID_LOAN — идентификатор кредита.

**D_close_loan**

Информация о статусах кредита (ссуд):
- ID_LOAN — идентификатор кредита;
- CLOSED_FL — текущий статус кредита (1 — закрыт, 0 — не закрыт).

Ниже представлен минимальный список колонок, которые должны находиться в итоговом датасете после склейки и агрегации данных. По своему усмотрению вы можете добавить дополнительные к этим колонки.

    - AGREEMENT_RK — уникальный идентификатор объекта в выборке;
    - TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);
    - AGE — возраст клиента;
    - SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
    - SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
    - GENDER — пол клиента (1 — мужчина, 0 — женщина);
    - CHILD_TOTAL — количество детей клиента;
    - DEPENDANTS — количество иждивенцев клиента;
    - PERSONAL_INCOME — личный доход клиента (в рублях);
    - LOAN_NUM_TOTAL — количество ссуд клиента;
    - LOAN_NUM_CLOSED — количество погашенных ссуд клиента.


Будьте внимательны при сборке датасета: это реальные банковские данные, в которых могут наблюдаться дубли, некорректно заполненные значения или значения, противоречащие друг другу. Для получения качественной модели необходимо предварительно очистить датасет от такой информации.

## Задание 1

В предыдущем задании вы собрали всю информацию о клиентах в одну таблицу, где одна строчка соответствует полной информации об одном клиенте.

Загрузите эту таблицу.

In [38]:
!pip install association-metrics -q
!pip install catboost -q

In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import association_metrics as am

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import make_pipeline

from catboost import CatBoostClassifier, Pool
import lightgbm as lgb
from lightgbm import LGBMClassifier, early_stopping, Dataset

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score, accuracy_score, classification_report, f1_score
from sklearn.metrics import roc_auc_score, roc_curve, auc, RocCurveDisplay

import warnings
warnings.filterwarnings("ignore")

import pickle

## Загрузка таблицы.

In [40]:
all_data = pd.read_csv('https://raw.githubusercontent.com/nedokormysh/lin_models_presentation/main/all_data.csv')
all_data.head()

,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,...,FAMILY_INCOME,PERSONAL_INCOME,LOAN_AMOUNT,CLOSED_LOANS,CREDIT,TERM,FST_PAYMENT,TARGET,WORK_TIME_IN_YEARS,LOAN_NOW
0,49,1,Среднее специальное,Состою в браке,2,1,1,0,Оренбургская область,Оренбургская область,...,от 10000 до 20000 руб.,5000.0,1,1,8000.00,6,8650.0,0,1.500000,0
1,32,1,Среднее,Состою в браке,3,3,1,0,Кабардино-Балкария,Кабардино-Балкария,...,от 10000 до 20000 руб.,12000.0,1,1,21650.00,6,4000.0,0,8.083333,0
2,52,1,Неполное среднее,Состою в браке,4,0,1,0,Иркутская область,Иркутская область,...,от 10000 до 20000 руб.,9000.0,2,1,33126.00,12,4000.0,0,7.000000,1
3,39,1,Высшее,Состою в браке,1,1,1,0,Ростовская область,Ростовская область,...,от 20000 до 50000 руб.,25000.0,1,1,8491.82,6,5000.0,0,14.000000,0
4,30,0,Среднее,Состою в браке,0,0,1,0,Кабардино-Балкария,Кабардино-Балкария,...,от 10000 до 20000 руб.,12000.0,2,1,21990.00,12,4000.0,0,8.416667,1


In [41]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13799 entries, 0 to 13798
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   AGE                      13799 non-null  int64  
 1   GENDER                   13799 non-null  int64  
 2   EDUCATION                13799 non-null  object 
 3   MARITAL_STATUS           13799 non-null  object 
 4   CHILD_TOTAL              13799 non-null  int64  
 5   DEPENDANTS               13799 non-null  int64  
 6   SOCSTATUS_WORK_FL        13799 non-null  int64  
 7   SOCSTATUS_PENS_FL        13799 non-null  int64  
 8   REG_ADDRESS_PROVINCE     13799 non-null  object 
 9   FACT_ADDRESS_PROVINCE    13799 non-null  object 
 10  POSTAL_ADDRESS_PROVINCE  13799 non-null  object 
 11  FL_PRESENCE_FL           13799 non-null  int64  
 12  OWN_AUTO                 13799 non-null  int64  
 13  GEN_INDUSTRY             13799 non-null  object 
 14  GEN_TITLE             

Это нужно было сделать в EDA ноутбуке, но в тот момент не реализовал это. Рассмотрим связь между категориальными признаками.

In [42]:
df_cat = all_data.apply(lambda x: x.astype("category") if x.dtype == "object" else x)
cramersv = am.CramersV(df_cat)

cramersv.fit()

,EDUCATION,MARITAL_STATUS,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,GEN_INDUSTRY,GEN_TITLE,JOB_DIR,FAMILY_INCOME
EDUCATION,1.000000,0.063281,0.101592,0.101397,0.102299,0.126464,0.146185,0.091051,0.109665
MARITAL_STATUS,0.063281,1.000000,0.102748,0.103022,0.102945,0.074834,0.048871,0.032852,0.131644
REG_ADDRESS_PROVINCE,0.101592,0.102748,1.000000,0.975258,0.988182,0.101547,0.107984,0.089879,0.255011
FACT_ADDRESS_PROVINCE,0.101397,0.103022,0.975258,1.000000,0.992647,0.101607,0.108238,0.094521,0.264824
POSTAL_ADDRESS_PROVINCE,0.102299,0.102945,0.988182,0.992647,1.000000,0.101411,0.106976,0.087068,0.262351
GEN_INDUSTRY,0.126464,0.074834,0.101547,0.101607,0.101411,1.000000,0.134249,0.131187,0.114742
GEN_TITLE,0.146185,0.048871,0.107984,0.108238,0.106976,0.134249,1.000000,0.097175,0.117245
JOB_DIR,0.091051,0.032852,0.089879,0.094521,0.087068,0.131187,0.097175,1.000000,0.068276
FAMILY_INCOME,0.109665,0.131644,0.255011,0.264824,0.262351,0.114742,0.117245,0.068276,1.000000


Видно, что адреса сильно коррелируют друг с другом. Поэтому будем использовать только почтовый адрес области.

In [43]:
# инициализируем листы числовых, категориальных, целевых признаков и признаки, которые не потребуются для анализа.
# cat_features_indxs = [0, 1, 2]

targets = ['TARGET']
features2drop = ['WORK_TIME', 'FACT_ADDRESS_PROVINCE', 'REG_ADDRESS_PROVINCE']
filtered_features = [i for i in all_data.columns if (i not in targets and i not in features2drop)]

continuous_features = ['CREDIT', 'FST_PAYMENT', 'AGE', 'CHILD_TOTAL', 'DEPENDANTS',
                       'OWN_AUTO', 'PERSONAL_INCOME', 'WORK_TIME', 'CLOSED_LOANS', 'LOAN_AMOUNT', 'WORK_TIME_IN_YEARS']
categorical_features = [i for i in all_data.columns if i not in continuous_features and (i not in targets and i not in features2drop)]

num_features = [i for i in filtered_features if i not in categorical_features]

In [44]:
for col in categorical_features:
    all_data[col] = all_data[col].astype("category")

## Разбиение данных на тренировочную и тестовую часть.

Разбейте данные на тренировочную и тестовую часть в пропорции 80% к 20%, зафиксируйте `random_state = 42`.

In [45]:
X = all_data[filtered_features].drop(targets, axis=1, errors="ignore")
y = all_data['TARGET']

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Прогнозирование целевой переменной.

На тренировочных данных обучите линейную модель классификации для предсказания целевой переменной (столбец `TARGET`).

Сделайте прогноз вероятности отклика на рекламную кампанию для тестовых данных.

In [47]:
# your code here

model = LogisticRegression(class_weight='balanced')

numeric_transformer = Pipeline(
                steps=[("imputer", SimpleImputer(strategy="median")),
                      ("scaler", StandardScaler())])
categorical_transformer = Pipeline(
                steps=[("encoder", OneHotEncoder(handle_unknown="ignore")),
                      #  ("selector", SelectPercentile(chi2, percentile=50)),
                      ])
preprocessor = ColumnTransformer(
                transformers=[("num", numeric_transformer, num_features),
                              ("cat", categorical_transformer, categorical_features)])

pipe = Pipeline([('feature_preprocessor', preprocessor),
                            ('model', model)])

## Метрики при стандартном пороге.

Переведите вероятности в классы по стандартному порогу (0.5) и на тестовом наборе данных вычислите метрики:

* accuracy
* precision
* recall
* f1-score

In [48]:
# your code here

pipe.fit(X_train, y_train)
probs = pipe.predict_proba(X_test)

probs_churn = probs[:,1]
classes = probs_churn > 0.5

In [49]:
# probs[:10]

In [50]:
# probs_churn[:10]

In [51]:
# classes[:10]

In [52]:
print(classification_report(y_test, classes))

              precision    recall  f1-score   support

           0       0.91      0.63      0.74      2390
           1       0.20      0.59      0.29       370

    accuracy                           0.62      2760
   macro avg       0.55      0.61      0.52      2760
weighted avg       0.81      0.62      0.68      2760



Целевая метрика для задачи - полнота, так как нам нужно найти максимум клиентов, кто может откликнуться на рекламу.

Но при этом точность не должна просесть, поэтому за ней тоже следим.

## Поиск оптимального порога разбиения.

Разбейте тренировочные данные на `train` и `val` части в пропорции 3 к 1.

В цикле:

* переберите пороги от 0 до 1 с шагом 0.01
* вычислите для каждого порога значение метрик precision и recall
* подберите такой порог, при котором recall не меньше 0.66, а точность максимальна.

In [53]:
X_train.shape

(11039, 23)

Разбиваем тренировочные данные.

Возможно в задании предполагалось, что мы ещё раз обучим модель, но уже на этом новом разбиении. Но вроде бы если строго следовать условию это не так. И мы просто получаем некую подвыборку y_val и смотрим результат в зависимости от порога.

In [54]:
X_train_n, X_val, y_train_n, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7575)

In [55]:
round(X_train_n.shape[0] / X_val.shape[0])

3

In [56]:
# model_mini = LogisticRegression(class_weight='balanced')
# pipe_mini = Pipeline([('feature_preprocessor', preprocessor),
#                             ('model', model_mini)])
# pipe_mini.fit(X_train_n, y_train_n)

# probs_mini = pipe_mini.predict_proba(X_val)
probs_mini = pipe.predict_proba(X_val)

precision_max = 0
best_res = {}
# перебираем пороги от 0 до 1 с шагом 0.01
for th in range(0, 101, 1):
    th = th if th == 0 else th/100
    probs_churn_mini = probs_mini[:,1]
    classes_mini = probs_churn_mini > th
    # вычисление метрик для каждого порога
    recall = recall_score(y_val, classes_mini)
    precision = precision_score(y_val, classes_mini)
    # print(f'th: {th} recall = {round(recall, 5)} precision = {round(precision, 5)}')

    # подбор порога при котором recall не меньше 0.66, а точность максимальна
    if recall >= 0.66:
        if precision > precision_max:
            precision_max = precision
            best_res['th'] = th
            best_res['recall'] = recall
            best_res['precision'] = precision_max
            # print(f'recall = {recall}, precision = {precision_max}, theshold = {th}')
print(f'Best result: th = {best_res["th"]}, recall = {round(best_res["recall"], 7)}, precision = {round(best_res["precision"], 7)}')

Best result: th = 0.5, recall = 0.6676829, precision = 0.1998175


Для выбранного порога посчитайте все метрики на тестовых данных. Сильно ли они отличаются от метрик на валидации?

In [57]:
# your code here

classes = probs_churn > best_res["th"]
print(classification_report(y_test, classes))

              precision    recall  f1-score   support

           0       0.91      0.63      0.74      2390
           1       0.20      0.59      0.29       370

    accuracy                           0.62      2760
   macro avg       0.55      0.61      0.52      2760
weighted avg       0.81      0.62      0.68      2760



К сожалению, в моих вычислениях, при том как я обработатал изначальные входные данные, порог 0.5 оказался оптимальным для данной задачи. Поэтому и результат не отличается от просто изначального результата.

## Выведем топ-6 признаков.

Выведите на экран в виде таблицы топ-6 признаков с наибольшими по модулю весами модели.

In [58]:
coef_table = pd.DataFrame({'features' : list(pipe[:-1].get_feature_names_out()), 'weights' : list(abs(model.coef_[0]))})

coef_table.sort_values(by='weights', ascending=False).head(6)

,features,weights
121,cat__GEN_INDUSTRY_Недвижимость,1.573913
131,cat__GEN_INDUSTRY_Страхование,1.518229
140,cat__GEN_INDUSTRY_Юридические услуги/нотариаль...,1.368344
65,cat__POSTAL_ADDRESS_PROVINCE_Мордовская респуб...,1.338297
181,cat__TERM_16,1.241759
24,cat__SOCSTATUS_WORK_FL_0,0.973183


Несколько неожиданно, но по моим результатам большие веса набрали категориальные признаки. Особенно связанные с работой.

Сохраним результаты.

In [59]:
with open('model.pickle', 'wb') as f:
    pickle.dump(pipe, f)

## Задание 2

Добавьте в Streamlit-приложение визуализацию результатов модели:

* опцию выбора порога и вывод метрик качества в зависимости от выбранного порога

* вывод прогноза модели на выбранном объекте (клиенте) - вероятность отклика на рекламу.

## Бонус

Попробуйте применить другие модели классификации для решения этой задачи (любые какие знаете).

Удалось ли добиться улучшения качества модели?

### Catboost

In [60]:
train_dataset = Pool(data=X_train, label=y_train,
                                 cat_features=categorical_features)
eval_dataset = Pool(data=X_test, label=y_test,
                                cat_features=categorical_features)

In [61]:
ctb_clf = CatBoostClassifier(eval_metric='Recall', auto_class_weights='Balanced')

ctb_clf.fit(train_dataset,
            eval_set=eval_dataset,
            verbose=100,
            early_stopping_rounds=400)

Learning rate set to 0.057347
0:	learn: 0.5411850	test: 0.5378378	best: 0.5378378 (0)	total: 62.9ms	remaining: 1m 2s
100:	learn: 0.7037572	test: 0.5864865	best: 0.6189189 (1)	total: 3.79s	remaining: 33.7s
200:	learn: 0.7557803	test: 0.5324324	best: 0.6189189 (1)	total: 11.2s	remaining: 44.5s
300:	learn: 0.8200867	test: 0.4594595	best: 0.6189189 (1)	total: 18.5s	remaining: 43s
400:	learn: 0.8605491	test: 0.4351351	best: 0.6189189 (1)	total: 26.8s	remaining: 40.1s
Stopped by overfitting detector  (400 iterations wait)

bestTest = 0.6189189189
bestIteration = 1

Shrink model to first 2 iterations.


In [62]:
y_pred = ctb_clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.53      0.67      2390
           1       0.17      0.62      0.27       370

    accuracy                           0.54      2760
   macro avg       0.53      0.57      0.47      2760
weighted avg       0.80      0.54      0.61      2760



In [63]:
confusion_matrix(y_test, y_pred)

array([[1266, 1124],
       [ 141,  229]])

Если не редактировать параметр auto_class_weights, то catboost почему-то у меня скатывался к предсказаниям из нулей.
Здесь я не использовал predict_proba, а оставил стандартный порог. И по recall метрике катбуст показал результаты лучше, чем стандартная логистическая регрессия. Параметры катбуста тоже не настраивал.

### LightGBM

In [98]:
X_le = X.copy()

le = LabelEncoder()
for col in categorical_features:
    X_le[col] = le.fit_transform(X_le[col])

In [99]:
X_le_train, X_le_test, y_le_train, y_le_test = train_test_split(X_le, y, test_size=0.2, random_state=42)

In [100]:
w = y_train.value_counts(normalize=False)[0] / y_train.value_counts(normalize=False)[1]

In [101]:
train_dataset = Dataset(X_le_train, y_le_train,
                        categorical_feature=categorical_features,
                        free_raw_data=False)

eval_dataset = Dataset(X_le_test, y_le_test,
                       categorical_feature=categorical_features,
                       free_raw_data=False,
                      #  weight=w
                       )

In [102]:
params = {'boosting_type': 'gbdt',
          'random_state': 42,
          'force_col_wise': True,
          'objective': 'binary',
          'scale_pos_weight': w
          }

In [103]:
lgbm_clf = lgb.train(params,
                     train_set=train_dataset,
                     valid_sets=(eval_dataset),
                     categorical_feature=categorical_features,
                     callbacks=[lgb.early_stopping(stopping_rounds=400)],
                     num_boost_round=400)

[LightGBM] [Info] Number of positive: 1384, number of negative: 9655
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 11039, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.125374 -> initscore=-1.942498
[LightGBM] [Info] Start training from score -1.942498
Training until validation scores don't improve for 400 rounds
Did not meet early stopping. Best iteration is:
[1]	valid_0's binary_logloss: 0.394023


In [104]:
probs_churn_lgbm = lgbm_clf.predict(X_le_test)
classes = probs_churn_lgbm > 0.5
print(classification_report(y_le_test, classes))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93      2390
           1       0.00      0.00      0.00       370

    accuracy                           0.87      2760
   macro avg       0.43      0.50      0.46      2760
weighted avg       0.75      0.87      0.80      2760



Стандартный порог опять же отнёс всё к нулевому классу. Подберём порог исходя из логики, близкой к тому, что мы делали выше.

In [105]:
precision_max = 0
best_res = {}

for th in range(0, 101, 1):
    th = th if th == 0 else th/100
    classes_lgbm = probs_churn_lgbm > th
    # вычисление метрик для каждого порога
    recall = recall_score(y_le_test, classes_lgbm)
    precision = precision_score(y_le_test, classes_lgbm)
    # print(f'th: {th} recall = {round(recall, 5)} precision = {round(precision, 5)}')

    # подбор порога при котором recall не меньше 0.66, а точность максимальна
    if recall >= 0.66:
        if precision > precision_max:
            precision_max = precision
            best_res['th'] = th
            best_res['recall'] = recall
            best_res['precision'] = precision_max
            # print(f'recall = {recall}, precision = {precision_max}, theshold = {th}')
print(f'Best result: th = {best_res["th"]}, recall = {round(best_res["recall"], 7)}, precision = {round(best_res["precision"], 7)}')

Best result: th = 0.16, recall = 0.7405405, precision = 0.1521377


In [106]:
probs_churn_lgbm = lgbm_clf.predict(X_le_test)
classes = probs_churn_lgbm > best_res["th"]
print(classification_report(y_le_test, classes))

              precision    recall  f1-score   support

           0       0.90      0.36      0.52      2390
           1       0.15      0.74      0.25       370

    accuracy                           0.41      2760
   macro avg       0.53      0.55      0.38      2760
weighted avg       0.80      0.41      0.48      2760



In [107]:
confusion_matrix(y_le_test, classes)

array([[ 863, 1527],
       [  96,  274]])

При помощи LightGBM удалось выжать recall на порядок лучше, чем в других моделях. (Вероятно, опять же, что в других моделях возможно были допущены ошибки.) Также ни в одних моделях не подбирались гиперпараметры, и не производились остальные манипуляции (возможно ансамблирование моделей) для улучшения метрики. Возможно ошибочно посчитал, что в рамках данного курса это не столь принципиально. Предполагал, что мы хотели больше сосредоточиться на демонстрации зависимости метрик от выбора порога.